# LSTM


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
from dataset import label_detection_training, label_detection_validation, silence_detection_training, silence_detection_validation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

In [2]:
results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

In [6]:
# Set your model's hyperparameters
for lstm_units in [64, 128, 256]:
    for dropout_rate in [0.2, 0.4, 0.6]:
            for epoch in [10, 20, 30, 40]:
                  for batch_size in [32, 64, 128]:

                        num_classes = 2  # Number of unique classes in your dataset

                        input_shape = (39, 44)

                        model = Sequential([
                            Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=input_shape),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(LSTM(lstm_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(32, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        history = model.fit(silence_detection_training.batch(batch_size), epochs=epoch, validation_data=silence_detection_validation.batch(batch_size))

                        results = np.concatenate((results, pd.DataFrame([[lstm_units, dropout_rate, epoch, batch_size, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


Epoch 1/10


ValueError: in user code:

    File "c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\keras\engine\training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\keras\engine\training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)


In [ ]:
results.to_pickle('results\\model_lstm')

array([[64, 0.2, 2, 32, 3.100442886352539,
        list([6.995808124542236, 3.100442886352539]),
        0.16634966433048248, 0.16634966433048248, 2.812389373779297,
        3.762976884841919, 0.18740805983543396, 0.18740805983543396]],
      dtype=object)

In [80]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])